# Multilayer Perceptron

Multilayer Perceptron (MLP) is a type of neural network that can be used for regression and classification.

This version of the workbook includes mini-batching added in 1.14 and momentum added in 1.15

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum Database 4.3.10.0
#%sql postgresql://gpdbchina@10.194.10.68:61000/madlib

u'Connected: gpadmin@madlib'

In [4]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.15-dev, git revision: rc/1.14-rc1-23-g5c4331d, cmake configuration time: Thu Jul 5 17:46:06 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# Classification without Mini-Batching

# 1.  Create input table for classification

In [5]:
%%sql 
DROP TABLE IF EXISTS iris_data;

CREATE TABLE iris_data(
    id serial,
    attributes numeric[],
    class_text varchar,
    class integer,
    state varchar
);

INSERT INTO iris_data(id, attributes, class_text, class, state) VALUES
(1,ARRAY[5.0,3.2,1.2,0.2],'Iris_setosa',1,'Alaska'),
(2,ARRAY[5.5,3.5,1.3,0.2],'Iris_setosa',1,'Alaska'),
(3,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Alaska'),
(4,ARRAY[4.4,3.0,1.3,0.2],'Iris_setosa',1,'Alaska'),
(5,ARRAY[5.1,3.4,1.5,0.2],'Iris_setosa',1,'Alaska'),
(6,ARRAY[5.0,3.5,1.3,0.3],'Iris_setosa',1,'Alaska'),
(7,ARRAY[4.5,2.3,1.3,0.3],'Iris_setosa',1,'Alaska'),
(8,ARRAY[4.4,3.2,1.3,0.2],'Iris_setosa',1,'Alaska'),
(9,ARRAY[5.0,3.5,1.6,0.6],'Iris_setosa',1,'Alaska'),
(10,ARRAY[5.1,3.8,1.9,0.4],'Iris_setosa',1,'Alaska'),
(11,ARRAY[4.8,3.0,1.4,0.3],'Iris_setosa',1,'Alaska'),
(12,ARRAY[5.1,3.8,1.6,0.2],'Iris_setosa',1,'Alaska'),
(13,ARRAY[5.7,2.8,4.5,1.3],'Iris_versicolor',2,'Alaska'),
(14,ARRAY[6.3,3.3,4.7,1.6],'Iris_versicolor',2,'Alaska'),
(15,ARRAY[4.9,2.4,3.3,1.0],'Iris_versicolor',2,'Alaska'),
(16,ARRAY[6.6,2.9,4.6,1.3],'Iris_versicolor',2,'Alaska'),
(17,ARRAY[5.2,2.7,3.9,1.4],'Iris_versicolor',2,'Alaska'),
(18,ARRAY[5.0,2.0,3.5,1.0],'Iris_versicolor',2,'Alaska'),
(19,ARRAY[5.9,3.0,4.2,1.5],'Iris_versicolor',2,'Alaska'),
(20,ARRAY[6.0,2.2,4.0,1.0],'Iris_versicolor',2,'Alaska'),
(21,ARRAY[6.1,2.9,4.7,1.4],'Iris_versicolor',2,'Alaska'),
(22,ARRAY[5.6,2.9,3.6,1.3],'Iris_versicolor',2,'Alaska'),
(23,ARRAY[6.7,3.1,4.4,1.4],'Iris_versicolor',2,'Alaska'),
(24,ARRAY[5.6,3.0,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(25,ARRAY[5.8,2.7,4.1,1.0],'Iris_versicolor',2,'Alaska'),
(26,ARRAY[6.2,2.2,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(27,ARRAY[5.6,2.5,3.9,1.1],'Iris_versicolor',2,'Alaska'),
(28,ARRAY[5.0,3.4,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(29,ARRAY[4.4,2.9,1.4,0.2],'Iris_setosa',1,'Tennessee'),
(30,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Tennessee'),
(31,ARRAY[5.4,3.7,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(32,ARRAY[4.8,3.4,1.6,0.2],'Iris_setosa',1,'Tennessee'),
(33,ARRAY[4.8,3.0,1.4,0.1],'Iris_setosa',1,'Tennessee'),
(34,ARRAY[4.3,3.0,1.1,0.1],'Iris_setosa',1,'Tennessee'),
(35,ARRAY[5.8,4.0,1.2,0.2],'Iris_setosa',1,'Tennessee'),
(36,ARRAY[5.7,4.4,1.5,0.4],'Iris_setosa',1,'Tennessee'),
(37,ARRAY[5.4,3.9,1.3,0.4],'Iris_setosa',1,'Tennessee'),
(38,ARRAY[6.0,2.9,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(39,ARRAY[5.7,2.6,3.5,1.0],'Iris_versicolor',2,'Tennessee'),
(40,ARRAY[5.5,2.4,3.8,1.1],'Iris_versicolor',2,'Tennessee'),
(41,ARRAY[5.5,2.4,3.7,1.0],'Iris_versicolor',2,'Tennessee'),
(42,ARRAY[5.8,2.7,3.9,1.2],'Iris_versicolor',2,'Tennessee'),
(43,ARRAY[6.0,2.7,5.1,1.6],'Iris_versicolor',2,'Tennessee'),
(44,ARRAY[5.4,3.0,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(45,ARRAY[6.0,3.4,4.5,1.6],'Iris_versicolor',2,'Tennessee'),
(46,ARRAY[6.7,3.1,4.7,1.5],'Iris_versicolor',2,'Tennessee'),
(47,ARRAY[6.3,2.3,4.4,1.3],'Iris_versicolor',2,'Tennessee'),
(48,ARRAY[5.6,3.0,4.1,1.3],'Iris_versicolor',2,'Tennessee'),
(49,ARRAY[5.5,2.5,4.0,1.3],'Iris_versicolor',2,'Tennessee'),
(50,ARRAY[5.5,2.6,4.4,1.2],'Iris_versicolor',2,'Tennessee'),
(51,ARRAY[6.1,3.0,4.6,1.4],'Iris_versicolor',2,'Tennessee'),
(52,ARRAY[5.8,2.6,4.0,1.2],'Iris_versicolor',2,'Tennessee');

SELECT * FROM iris_data ORDER BY id;

Done.
Done.
52 rows affected.
52 rows affected.


id,attributes,class_text,class,state
1,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


#  2. Classification model

Generate a multilayer perceptron with a single hidden layer of 5 units. Use the attributes column as the independent variables, and use the class column as the classification. Set the tolerance to 0 so that 500 iterations will be run. Use a hyperbolic tangent activation function. The model will be written to mlp_model.

In [6]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=500,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model:

In [7]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[-0.241222466342704, 0.249761165436226, 0.0303614415534361, 0.175507685036152, 0.0815734316744983, -0.139212167184273, -0.523351688827021, 0.682480785215137, -0.621908347655988, -0.469472551411455, 0.0147756731570967, -0.147278260748286, 0.305555593978066, -0.56090735505522, -0.161234544160645, 0.271024873678476, 0.353270530669186, -0.584992635094265, 0.805209136021446, 0.680671510984583, 0.263358536565038, 0.416395782728773, -0.536749491420233, 0.401066730627837, 0.851835204311541, -0.118513505811856, -0.0923782150443473, 1.08817076327241, 0.581060997166745, -1.65259364782558, -1.05618327045111, 0.319893429079889, 0.227676950437848, -1.23886857988594, -0.403615235762293, 1.2788824239611, 1.35644018931145]",0.000759051397192,500


View the model summary table:

In [8]:
%%sql
SELECT * FROM mlp_model_summary;

1 rows affected.


source_table,independent_varname,dependent_varname,dependent_vartype,tolerance,learning_rate_init,learning_rate_policy,momentum,nesterov,n_iterations,n_tries,layer_sizes,activation,is_classification,classes,weights,grouping_col
iris_data,attributes,class_text,character varying,0.0,0.003,constant,0.9,True,500,1,"[4, 5, 2]",tanh,True,"[u'Iris_setosa', u'Iris_versicolor']",1,NULL


View the standardization table:

In [9]:
%%sql
SELECT * FROM mlp_model_standardization;

1 rows affected.


mean,std
"[5.45961538461539, 2.99807692307692, 3.025, 0.851923076923077]","[0.598799958694505, 0.498262513685689, 1.41840579525043, 0.550346179381454]"


# 3. Prediction for classification

Now let's use the model to predict. In the following example we will use the training data set for prediction as well, which is not usual but serves to show the syntax. The prediction is in the estimated_class_text column with the actual value in the class_text column.

In [10]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model',         -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (id) ORDER BY id;

Done.
1 rows affected.
52 rows affected.


id,estimated_class_text,attributes,class_text,class,state
1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


Count the misclassifications:

In [11]:
%%sql
SELECT COUNT(*) FROM mlp_prediction JOIN iris_data USING (id) 
WHERE mlp_prediction.estimated_class_text != iris_data.class_text;

1 rows affected.


count
0


# Classification with Mini-Batching

# 1. Call mini-batch preprocessor

Use the same data set as above.

In [12]:
%%sql
DROP TABLE IF EXISTS iris_data_packed, iris_data_packed_summary, iris_data_packed_standardization;

SELECT madlib.minibatch_preprocessor('iris_data',         -- Source table
                                     'iris_data_packed',  -- Output table
                                     'class_text',        -- Dependent variable
                                     'attributes'        -- Independent variables
                                     );

SELECT * FROM iris_data_packed ORDER BY __id__;

Done.
1 rows affected.
2 rows affected.


__id__,dependent_varname,independent_varname
0,"[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0]]","[[0.0674425821070736, -1.20032494247982, 0.475886380512869, 0.269061417385597], [-0.767560815504508, -2.00311461461654, 0.334883008509056, 0.269061417385597], [-0.433559456459875, -0.598232688377286, 0.616889752516682, 0.995876674738521], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [0.568444620674023, -0.798930106411465, 0.687391438518589, 0.632469046062059], [-0.767560815504508, 0.806649237861967, -1.07515071152907, -1.18456909732025], [-0.934561495026824, -1.20032494247982, 0.193879636505243, 0.269061417385597], [0.902445979718656, -0.196837852308928, 1.03989986852812, 1.17758048907675], [0.0674425821070736, 1.00734665589615, -1.21615408353289, -1.18456909732025], [1.23644733876329, -1.60171977854818, 1.03989986852812, 1.17758048907675], [1.06944665924097, 0.00385956572525086, 1.11040155453003, 0.995876674738521], [-1.76956489263841, 0.00385956572525086, -1.21615408353289, -1.18456909732025], [1.4034480182856, 0.605951819827788, 1.18090324053193, 1.35928430341498], [1.4034480182856, -1.401022360514, 0.969398182526215, 0.81417286040029], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [-0.0995580974152422, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [-1.76956489263841, 0.405254401793609, -1.21615408353289, -1.18456909732025], [-1.93656557216072, 0.00385956572525086, -1.3571574555367, -1.36627291165848], [0.902445979718656, 0.806649237861967, 1.03989986852812, 1.35928430341498], [-0.767560815504508, 1.00734665589615, -1.00464902552717, -0.457753839967327], [-1.10156217454914, 0.00385956572525086, -1.14565239753098, -1.36627291165848], [-1.60256421311609, -1.401022360514, -1.21615408353289, -1.00286528298202], [0.401443941151707, 2.81362341820376, -1.07515071152907, -0.821161468643789], [-0.767560815504508, 1.00734665589615, -1.21615408353289, -1.00286528298202], [0.568444620674023, -0.598232688377286, 0.757893124520495, 0.269061417385597], [0.568444620674023, 2.01083374606704, -1.28665576953479, -1.18456909732025]]"
1,"[[1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0]]","[[-1.76956489263841, -0.196837852308928, -1.14565239753098, -1.18456909732025], [0.401443941151707, -0.397535270343108, 1.03989986852812, 0.81417286040029], [0.902445979718656, -1.60171977854818, 0.687391438518589, 0.269061417385597], [0.234443261629389, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [-0.600560135982193, 1.60943890999868, -0.793143967521448, -0.821161468643789], [-0.600560135982193, 0.806649237861967, -1.07515071152907, -1.18456909732025], [0.401443941151707, -0.798930106411465, 0.334883008509056, 0.269061417385597], [0.73544530019634, 0.00385956572525086, 0.828394810522402, 1.17758048907675], [-0.600560135982193, 1.60943890999868, -1.00464902552717, -1.18456909732025], [1.06944665924097, -0.196837852308928, 1.18090324053193, 0.995876674738521], [0.234443261629389, -0.196837852308928, 0.405384694510963, 0.81417286040029], [-0.767560815504508, 0.405254401793609, -1.28665576953479, -1.18456909732025], [0.0674425821070736, -0.999627524445644, 0.687391438518589, 0.81417286040029], [0.902445979718656, -0.598232688377286, 1.46290998453956, 1.35928430341498], [0.568444620674023, -0.598232688377286, 0.616889752516682, 0.632469046062059], [-0.0995580974152422, 1.81013632803286, -1.21615408353289, -0.821161468643789], [1.90445005685255, -0.1968378523089

# 2.  Classification model with mini-batching
Use similar parameters as before:

In [13]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;
-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data_packed',      -- Ouput table from mini-batch preprocessor
    'mlp_model',             -- Destination table
    'independent_varname',   -- Hardcode to this, from table iris_data_packed
    'dependent_varname',     -- Hardcode to this, from table iris_data_packed
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.1,
    n_iterations=500,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model:

In [14]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[-0.13252534498649, -0.37896100676424, 0.503403120152183, -1.08679816249876, -0.561814480613823, 0.402548045485862, 0.532705053534283, -0.552630848296953, 0.67855516747366, 0.413920498590441, -0.256766228396609, -0.310783793721915, 0.473025380015013, -0.764847565888928, -0.633685437636607, 0.331118371614658, 0.68785202110586, -0.730905401730095, 0.812276423458694, 1.0028080447692, -0.247150080195493, -0.326479660450041, 0.272027165497577, -0.783484532590993, -0.3240005914012, -0.292052260754316, 0.942848112860559, -0.611370901320803, 0.727726656356208, -1.80532276028114, 0.859042273992032, 0.245507590486603, -1.36472202886508, 0.750248279831834, -0.892308360996602, 1.34236183516907, -0.528363560505027]",0.000217511008854,500


# 3. Prediction for classification

Now predict using model we built using mini-batching.  As before we will use the training data set for prediction as well, which is not usual but serves to show the syntax. The prediction is in the estimated_class_text column with the actual value in the class_text column.  Note that the prediction function is exactly the same whether you use mini batching or not:

In [15]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model',         -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (id) ORDER BY id;

Done.
1 rows affected.
52 rows affected.


id,estimated_class_text,attributes,class_text,class,state
1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


Count the missclassifications:

In [16]:
%%sql
SELECT COUNT(*) FROM mlp_prediction JOIN iris_data USING (id) 
WHERE mlp_prediction.estimated_class_text != iris_data.class_text;

1 rows affected.


count
0


# Classification with Other Parameters

# 1.  Classification with n_tries
Now, use the n_tries optimizer parameter to learn and choose the best model among n_tries number of models learned by the algorithm. Run only for 50 iterations and choose the best model from this short run.  Note we are not using mini-batching here.

In [17]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=50,
    tolerance=0,
    n_tries=3',       -- Optimizer params, with n_tries
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model:

In [18]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[0.0244555048721247, -0.0102220957795577, -0.296304368151928, 0.417354520714385, 0.716356145927118, 0.136764692058528, 0.171990003420157, -0.201210208125569, 0.77035771901118, 0.407708752162676, -0.271990858018259, -0.110001207558304, 0.0764625013336321, -0.662551454607256, -0.345752007670019, -0.0257387656399257, 0.0163162479352703, 0.141741825035307, -0.194861547764694, -0.507526714801008, -0.198900918139106, -0.266562880474682, 0.504729117752748, -0.703364999113683, -0.504193062582795, -0.044388781231599, -0.512089695364596, -0.908221978560789, 0.377288755491792, 0.533781829174235, 0.95222971088937, 0.278101830799836, 0.849808209884716, 0.657034958332216, -0.717554398768694, -0.161169131664735, -1.16498798580801]",0.00769081296841,50


# 2. Classification with warm start
Next, use the warm_start parameter to start learning a new model, using the coefficients already present in mlp_model. Note that we must not drop the mlp_model table, and cannot use the n_tries parameter if warm_start is used.

In [19]:
%%sql
SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=450,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    TRUE,             -- Warm start
    FALSE             -- Not verbose
);

1 rows affected.


mlp_classification
""


View the classification model with warm_start.  The algorithm learnt three different models starting with a different set of initial weights for the coefficients, and chose the best model among them.

In [20]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[0.082253254319968, 0.0471209370913246, -0.384821317561673, 0.526418188039721, 0.817005881365742, 0.202236335415648, 0.196782764147218, -0.317083296110335, 0.868670196201861, 0.5023642759333, -0.297707081940346, -0.15474036564037, 0.189877667636285, -0.772636865741197, -0.450007756608428, -0.0618953340072429, -0.0485493692670159, 0.233726227905363, -0.309319300090572, -0.614427690931041, -0.241943033066046, -0.308559081483271, 0.573255747101207, -0.800908414588587, -0.59797357646535, -0.0655114377779145, -0.757449508625442, -1.16029109905661, 0.611301307133223, 0.726156396360849, 1.22128945862313, 0.299224487346315, 1.09516802314545, 0.90910407882761, -0.951566950410224, -0.353543698851853, -1.43404773354414]",0.000622333975641,450


# Classification with grouping


# 1. Train classification model
Group the training data by state, and learn a different model for each state.  Note we are not using mini-batching in this example.

In [21]:
%%sql
DROP TABLE IF EXISTS mlp_model_group, mlp_model_group_summary, mlp_model_group_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',        -- Source table
    'mlp_model_group',  -- Destination table
    'attributes',       -- Input features
    'class_text',       -- Label
    ARRAY[5],           -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=500,
    tolerance=0',       -- Optimizer params
    'tanh',             -- Activation function
    NULL,               -- Default weight (1)
    FALSE,              -- No warm start
    FALSE,              -- Not verbose
    'state'             -- Grouping column
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model with grouping.  A separate model is learnt for each state, and the result table displays the name of the state (grouping column) associated with the model.

In [22]:
%%sql
SELECT * FROM mlp_model_group ORDER BY state;

2 rows affected.


state,coeff,loss,num_iterations
Alaska,"[0.0517171189458982, 0.434746538896262, -0.380359689123331, 0.461023338048344, 0.276313665483947, 0.333652919771562, 0.543526053438161, -0.579790476005051, 0.631019705485354, 0.77787063008935, -0.267185488788932, -0.215376830980525, 0.142023838279892, -0.0980153714373588, 0.025962444419142, 0.145614982983637, 0.226511840535411, -0.428940694265705, 0.489553694802101, 0.920812531260085, 0.0976621626919583, 0.208323991800446, -0.457175616382581, 0.689510975725241, 0.396240966367252, 0.0190565236797046, -0.730155173532018, -1.30384064046762, 0.130113100235692, -1.02602491180251, -0.805263046903185, 0.223419892178926, 0.539725947692049, 1.5916702825289, -0.157765353227868, 1.22477724859327, 0.863002777392142]",0.00149241520199,500
Tennessee,"[-0.0719489568363201, -0.00846859240152588, -0.245989310035623, 0.0906051433426646, -0.285296413231384, -0.272912515784428, -0.365126864442536, 0.506780999273071, -0.643857920196186, -0.598082920020583, -0.334289774126983, -0.240709845942713, 0.416327176691397, -0.707129785373748, -0.406758117698223, 0.226771563366074, 0.239264065458081, -0.453912069894399, 0.816740348800308, 0.279973297559166, 0.372662883118607, 0.240314273369794, -0.273084457739955, 0.337982886929062, 0.857700076232814, -0.223599346924905, 0.0401280034806427, 1.30110268406518, 0.923553219386498, -0.917084713252914, -0.811101744745542, -0.0428446364822329, -0.0033944616934443, -1.04634323088737, -0.991594893022902, 0.946629750594729, 1.05140657321989]",0.0014374967735,500


# 2. Prediction for classification using grouping
Prediction based on grouping using the state column:

In [23]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model_group',   -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (state,id) ORDER BY state, id;

Done.
1 rows affected.
52 rows affected.


state,id,estimated_class_text,attributes,class_text,class
Alaska,1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1
Alaska,2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1
Alaska,4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1
Alaska,6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1
Alaska,7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1
Alaska,8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1
Alaska,10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1


# Regression without Mini-Batching

# 1. Create input table for regression
This dataset contains housing prices data, where zipcode can be used as a grouping column.

In [24]:
%%sql
DROP TABLE IF EXISTS lin_housing;

CREATE TABLE lin_housing (id serial, 
                          x numeric[], 
                          zipcode int, 
                          y float8);

INSERT INTO lin_housing(id, x, zipcode, y) VALUES
(1,ARRAY[1,0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98],94016,24.00),
(2,ARRAY[1,0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14],94016,21.60),
(3,ARRAY[1,0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03],94016,34.70),
(4,ARRAY[1,0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94],94016,33.40),
(5,ARRAY[1,0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6.0622,3,222.0,18.70,396.90,5.33],94016,36.20),
(6,ARRAY[1,0.02985,0.00,2.180,0,0.4580,6.4300,58.70,6.0622,3,222.0,18.70,394.12,5.21],94016,28.70),
(7,ARRAY[1,0.08829,12.50,7.870,0,0.5240,6.0120,66.60,5.5605,5,311.0,15.20,395.60,12.43],94016,22.90),
(8,ARRAY[1,0.14455,12.50,7.870,0,0.5240,6.1720,96.10,5.9505,5,311.0,15.20,396.90,19.15],94016,27.10),
(9,ARRAY[1,0.21124,12.50,7.870,0,0.5240,5.6310,100.00,6.0821,5,311.0,15.20,386.63,29.93],94016,16.50),
(10,ARRAY[1,0.17004,12.50,7.870,0,0.5240,6.0040,85.90,6.5921,5,311.0,15.20,386.71,17.10],94016,18.90),
(11,ARRAY[1,0.22489,12.50,7.870,0,0.5240,6.3770,94.30,6.3467,5,311.0,15.20,392.52,20.45],94016,15.00),
(12,ARRAY[1,0.11747,12.50,7.870,0,0.5240,6.0090,82.90,6.2267,5,311.0,15.20,396.90,13.27],20001,18.90),
(13,ARRAY[1,0.09378,12.50,7.870,0,0.5240,5.8890,39.00,5.4509,5,311.0,15.20,390.50,15.71],20001,21.70),
(14,ARRAY[1,0.62976,0.00,8.140,0,0.5380,5.9490,61.80,4.7075,4,307.0,21.00,396.90,8.26],20001,20.40),
(15,ARRAY[1,0.63796,0.00,8.140,0,0.5380,6.0960,84.50,4.4619,4,307.0,21.00,380.02,10.26],20001,18.20),
(16,ARRAY[1,0.62739,0.00,8.140,0,0.5380,5.8340,56.50,4.4986,4,307.0,21.00,395.62,8.47],20001,19.90),
(17,ARRAY[1,1.05393,0.00,8.140,0,0.5380,5.9350,29.30,4.4986,4,307.0,21.00,386.85,6.58],20001, 23.10),
(18,ARRAY[1,0.78420,0.00,8.140,0,0.5380,5.9900,81.70,4.2579,4,307.0,21.00,386.75,14.67],20001,17.50),
(19,ARRAY[1,0.80271,0.00,8.140,0,0.5380,5.4560,36.60,3.7965,4,307.0,21.00,288.99,11.69],20001,20.20),
(20,ARRAY[1,0.72580,0.00,8.140,0,0.5380,5.7270,69.50,3.7965,4,307.0,21.00,390.95,11.28],20001,18.20);

SELECT * FROM lin_housing ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,x,zipcode,y
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9


# 2. Train regression model
Now train a regression model using a multilayer perceptron with 2 hidden layers of 25 nodes each.

In [25]:
%%sql
DROP TABLE IF EXISTS mlp_regress, mlp_regress_summary, mlp_regress_standardization;

SELECT setseed(0);
SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


View the regression model:

In [26]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([-0.656383359593314, 0.0981942667462611, 0.15088933926421, -0.106633987538081, -0.0899687952809924, -0.0145333834617717, 0.12136378081547, -0.782819122944032, 0.259671128064098, -0.502780551085695, 0.171853608947973, 0.131923038813867, 0.107423643861035, -0.22169097701093, 0.662099530018031, -0.334496419741836, 0.102396026147682, 0.00797396264060859, -0.101067195641826, -0.030228585509483, 0.000165733784327993, 0.12678814124639, -0.46672350141396, 0.141344091161405, -0.331672256057709, -0.0153138767258305, 0.0268856612939619, 0.0356245784465816, -0.0978578292658108, 0.446919524889663, -1.07038054016061, 0.0284313661506061, 0.498894731852639, 0.400998261009715, 0.110629658757457, -0.0323232913091121, 1.11838081720475, -0.377037128364206, -0.314089859403417, 0.213680759499323, 0.936161964208252, 0.835821283918469, 0.167522766560203, 0.111762035835106, -0.182798269179843, -1.02010118832038, 0.144715612938809, 0.443906722792342, 0.54669801140408, -0.0782577910629485, 0.0388648637586384, 1.39953876924091, -0.345064509968093, -0.180387395231858, -0.148777549918212, 1.09231129386625, 1.02911231438098, 0.0919058161473731, -0.0279149768225324, -0.52200624599166, -0.345064650514297, -0.0305820836479825, 0.30544548757515, 0.264678309747157, 0.0554464320770866, 0.00588471544033355, 0.849522095509909, 0.219076403934794, -0.185662244329109, -0.17085630746285, 0.751215402232775, 0.601261000768835, 0.0937848616097991, -0.0968903182976813, -0.180155387544642, -0.28693318548236, -0.0414793818079976, -0.048463962861907, 0.46708139223447, 0.302867513819116, -0.0386228008896532, 1.07368461176052, -0.408074368114306, -0.271467384138417, -0.439050479113033, 0.941670742675273, 0.980059187664242, 0.0392541333230186, -0.0974446236586582, -0.695028180477456, -0.684552728885673, -0.0530504570677959, -1.23711121808766, -0.0303566371335746, -0.91841557357917, -0.0480387469971307, -0.391854726710006, -0.727495426568053, 1.67195258802171, -1.48505303415493, 0.053018119152746, -0.475550098642568, -0.293109768714992, 0.968505211912189, 0.371544632701374, 0.191129852295399, -0.0208553296335499, -0.379311192768689, 0.200075124396837, 0.0943038615947207, 0.0822366873616073, 0.205314834737529, -0.743754110014029, -0.0211855006877727, -0.498851709982437, 0.221105084124169, 0.122068926443212, -0.237249974623739, 0.384991696870278, -0.918082774570148, 0.0474016188476247, -0.0712642118774827, -0.114737530019968, -0.0617064058085067, 0.159256558409089, -0.0774295953923433, -0.0303634231571478, -0.20427086955533, -0.025743476619476, 0.0797625743378702, 0.241576941424533, 0.12054807167765, 0.121721340969692, 0.296161274355386, -0.21484032183402, -0.38701841657614, 0.0791154842327235, 0.190430807186172, 0.346669418625634, 0.0137187119481801, 0.0537189704502057, 0.70245683240513, -0.0445769309212651, -0.124425546744604, -0.189658692452825, 0.540030057618568, 0.460244988557846, 0.049046294913715, 0.00979319598231975, -0.211222273600647, -0.329067613447065, -0.00548846108559954, -0.413343887858384, -0.139982454329231, 0.31447494335565, -0.0753501869794823, 0.128496337590319, 0.166577232680209, 0.551790998403117, -0.863679393049108, 0.593079970631553, 0.0497138793098114, 0.365313778352169, 0.935240235598543, 0.00793715259295871, -0.899122803382757, 0.0921855466169052, -0.931810959407966, 0.259083719423246, 0.00518402049063279, 0.07053296349812, -0.134491815246468, 0.318679022264602, 0.758389200850775, -1.64610923768458, 0.825137571745062, 0.0284267093681851, -0.308688103634016, 0.65665159483004, -0.0492139205843407, -0.00922487318890386, 0.0184594257792922, -0.144198990427024, 0.194347108771803, 0.178767586624165, 0.114454970917256, 0.313338749883121, -0.2211232793894, -0.209994770290604, -0.120815838119693, 0.422062201863381, 0.467691346141436, 0.0548853406048129, -0.0547738601840598, -0.346290874671382, -0.680572886471843, 0.0383853185577226, 0.211850195638899, 0.363027606690681, 0.0260654554553261, 0.0095933109055222, 0.7781099594393, -0.333035768078405, -0.2198127237642

# 3.  Prediction for regression

In [27]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress',               -- Model table
         'lin_housing',               -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT *, ABS(y-estimated_y) as abs_diff FROM lin_housing JOIN mlp_regress_prediction USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,x,zipcode,y,estimated_y,abs_diff
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0,23.9468470783,0.0531529216702
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6,21.6302588798,0.0302588797719
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7,34.6125433183,0.0874566817047
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4,34.6125433183,1.2125433183
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2,34.6125433183,1.5874566817
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7,28.7695264856,0.0695264856477
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9,22.8948242665,0.00517573348926
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1,26.9649740369,0.135025963131
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5,16.500596278,0.000596277991971
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9,18.9140968013,0.0140968012766


RMS error:

In [31]:
%%sql
SELECT SQRT(AVG((y-estimated_y)*(y-estimated_y))) as rms_error FROM lin_housing 
JOIN mlp_regress_prediction USING (id);

1 rows affected.


rms_error
0.451761607985


# Regression with Mini-Batching

# 1. Call mini-batch preprocessor

Call mini-batch preprocessor using the same data set as above:

In [32]:
%%sql
DROP TABLE IF EXISTS lin_housing_packed, lin_housing_packed_summary, lin_housing_packed_standardization;

SELECT madlib.minibatch_preprocessor('lin_housing',         -- Source table
                                     'lin_housing_packed',  -- Output table
                                     'y',                   -- Dependent variable
                                     'x'                   -- Independent variables
                                     );

SELECT * FROM lin_housing_packed ORDER BY __id__;

Done.
1 rows affected.
2 rows affected.


__id__,dependent_varname,independent_varname
0,"[[18.9], [18.9], [36.2], [21.7], [34.7], [20.2], [28.7], [27.1], [20.4], [19.9]]","[[0.0, -0.631770483117677, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.389524486713741, 0.776185297991884, 1.12338173490388, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.421245698698072, 0.266062209630877], [0.0, -0.471896726029508, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.400175962970612, 0.926706040976735, 1.53190044583817, 0.998114984186569, 0.652271651376812, -1.12035390762777, -0.0215533181921044, 0.856455177699393], [0.0, -0.779023428109291, -0.803851813725231, -1.9969773137395, 0.0, -1.8347407829862, 2.03475150934991, -0.663796476563195, 0.939470054694005, -0.737737162224855, -1.93934339203999, 0.270156261413788, 0.421245698698072, -0.957885614511164], [0.0, -0.703815552651988, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.645159916878625, -1.42643490768644, 0.256034126959662, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.143138367187069, 0.642187233674527], [0.0, -0.90602225055559, -0.803851813725231, 0.138430310180499, 0.0, -1.47562781834477, 2.11570272890212, -0.317598767698037, -0.284856273560335, -1.60566323543057, -1.35695798902498, -0.0874034963397545, 0.244386817565293, -1.15828009453442], [0.0, 1.45215368098917, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -1.56757776072358, -1.54685150207432, -1.59359191400827, 0.130188910980857, 0.53579457077381, 1.18392008678395, -4.26790463620074, 0.0225058416026117], [0.0, -0.898236882129764, -0.803851813725231, -1.9969773137395, 0.0, -1.8347407829862, 0.507329814114726, -0.438015362085918, 0.939470054694005, -0.737737162224855, -1.93934339203999, 0.270156261413788, 0.300442826572981, -0.976383566513311], [0.0, -0.549415882738228, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.0422863607397744, 1.43847656712523, 0.814588992812889, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.421245698698072, 1.17246185773607], [0.0, 0.926185138314665, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.517342201796184, -0.282477261001572, -0.575090147010091, 0.130188910980857, 0.53579457077381, 1.18392008678395, 0.421245698698072, -0.506227286458749], [0.0, 0.918977590201693, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.762326155704198, -0.548397240274809, -0.808641210814613, 0.130188910980857, 0.53579457077381, 1.18392008678395, 0.365624232395872, -0.473855870454992]]"
1,"[[18.2], [22.9], [18.2], [33.4], [17.5], [21.6], [23.1], [15.0], [16.5], [24.0]]","[[0.0, 1.21825810066482, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.990267747601218, 0.103859312659547, -1.59359191400827, 0.130188910980857, 0.53579457077381, 1.18392008678395, 0.162692788933936, -0.040695494404723], [0.0, -0.720511518533937, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.38313360095962, -0.0416440722258095, 0.378567380157248, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.364755146984901, 0.13657654561585], [0.0, 0.951122646553642, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.2041887998442, 0.856463027583804, -0.849671962569644, 0.130188910980857, 0.53579457077381, 1.18392008678395, -0.3122623881622, -0.19792808642297], [0.0, -0.890573160085591, -0.803851813725231, -1.9969773137395, 0.0, -1.8347407829862, 1.71733751689518, -1.08525455692078, 0.939470054694005, -0.737737162224855, -1.93934339203999, 0.270156261413788, 0.322604504552763, -1.32630315855392], [0.0, 1.395861817879, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.430000096489847, 0.715977000797943, -1.07774480611259, 0.130188910980857, 0.53579457077381, 1.18392008678395, -0.0198151473701598, 0.481871649655922], [0.0, -0.905961427364764, -0.803851813725231, 0.138430310180499, 0.0, -1.47562781834477, 0.488157156852361, 0.575490974012082, -0.284856273560335, -1.60566323543057,

# 2. Train regression model with mini-batching

In [33]:
%%sql
DROP TABLE IF EXISTS mlp_regress, mlp_regress_summary, mlp_regress_standardization;

SELECT setseed(0);
SELECT madlib.mlp_regression(
    'lin_housing_packed',    -- Source table
    'mlp_regress',           -- Desination table
    'independent_varname',   -- Hardcode to this, from table lin_housing_packed
    'dependent_varname',     -- Hardcode to this, from table lin_housing_packed
    ARRAY[25,25],            -- Number of units per layer
    'learning_rate_init=0.01,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',            -- Optimizer params
    'tanh',                  -- Activation function
    NULL,                    -- Default weight (1)
    FALSE,                   -- No warm start
    FALSE                    -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


View model:

In [34]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([-0.435946003312333, -0.145186148819241, 0.38582547315735, -0.0608519787318397, 0.6730543753469, 0.0177064230623991, 0.724959269635173, -0.564093496987584, 0.208037115095767, -0.0271620369275532, 0.58038352727916, 0.516854218839299, 0.195071226643122, -0.11564161735909, 0.261477197155572, -0.185246843623561, -0.128136851775235, -1.04368113961963, 0.222021249238062, -0.436040368102035, -0.0294913179365429, -0.487709533381466, 0.337535060637899, 1.05973672564886, -1.43621973044461, -0.289698366847503, -0.411657112803319, -0.192451466039826, 0.661873796033307, 0.0154955201655609, 0.508582687101413, 0.0971875971231677, -0.215543636763707, -0.0132300219347536, -0.417212321853377, -0.0160392595676297, -0.400734130056441, 0.0345349995516341, -0.422524131974013, 0.0478504376257955, -0.322065077661565, -0.350749844241257, -0.0288221450799517, -0.141844901026121, -0.226409619713839, -0.180075237590242, -0.00259264692179361, 0.236245944281006, -0.0942367819984068, -0.20271232437243, -0.104979501281054, -0.180260718457536, 0.279993995114081, -0.432089104267163, 1.46474491778129, -0.110513132356017, -0.0996900032337406, -0.0950768500055085, -0.200856170497519, -0.157550371335665, -0.977266476032198, 0.120321643820273, 0.00437431734059586, -0.269184166896661, -0.143440222988864, -0.0332306383769334, -0.0066111596109172, 0.829090032867279, -0.723273232082523, 0.0571080187939085, 0.341943587938623, -0.0355206212924825, 0.368449391006724, -0.0397549620668309, -0.640026582436604, -0.0143858555815269, 0.0634744114809924, -0.18492321574482, -0.248989598615352, -0.742446272609742, -0.019226649750434, -0.766489518154356, 0.667091405673553, -0.549241595557052, -0.179614650233818, -0.790268166118861, -0.755990232433582, -0.0379841914003529, -0.0801648874483325, -0.537552966342003, 1.22386252362828, -0.0364796318694084, -0.285792544409613, 0.0563433466023855, -0.466703462451257, -0.155793202494595, -0.445802225143066, 0.228839931267574, -0.447151144298666, -0.223608388189774, -0.514991494811221, -0.652594058034304, -0.109683673446235, -0.0998752214087438, -0.27965146457009, 0.0455791511176733, 0.0491789033831562, 0.150083657253015, -0.0462484238948723, 0.319274907190568, -0.0183689296900765, 0.131602545414316, -0.419860914405633, -0.0972171098783859, 0.0804402081956685, 0.312112375634662, 0.222042032827657, -0.104695231716597, -0.215057582849687, 0.0691788044660947, -0.612526086751577, 0.0854968627288611, -0.00119213359912633, 0.304761252633513, 0.73156928787095, -0.134179297919786, 0.622111212343324, -0.373206230595744, 0.848372880399822, 0.465675768128763, 0.744870045475251, 0.676267459673554, -0.0307823875857259, 0.748140541792685, 0.520983067089359, 0.732915170668436, 0.0574620721093038, -0.31997761717281, 0.223241736286222, 0.19377078631622, -0.00515647446081029, -0.0512296220488033, -0.669181515425964, 0.721138513490966, 0.13177361517086, -0.369483113032403, 0.0336972823296043, -0.251406300878073, 0.0229662826803323, 0.565306193228724, -0.663982029118315, 0.0660834464393142, 0.288014281525032, -0.0525242702306875, 0.424875386705359, -0.0270342580217193, 0.528489059322906, -0.211949393629478, 0.817896966954087, 0.104804222319907, 0.442541908867057, 0.422588487018307, 0.15767441057888, 0.431095264102292, 0.103485211358293, 0.166315267469171, -0.0900257923762943, -0.123476956884149, 0.0412676029262655, -0.106029586627453, -0.0292444940152174, -0.177835398711136, 0.0338952847869845, -0.230559408935971, -0.0469685738855755, 0.0291820265004914, -0.208477284700808, -0.140036226360732, 0.00779708488413704, 0.0445137626952362, -0.54816937914994, -0.0427816264305856, 0.148550212528845, 0.348217810683318, 0.570285144425328, 0.0478517821943428, 0.523627035904701, -0.430985083287075, 0.772613826964201, 0.201592277485111, 0.537361833429133, 0.679619449426633, 0.0323521685563921, 0.388518165128797, 0.362277058109766, 0.638004045657666, -0.0672150430556288, -0.306158108921929, 0.259122754366641, 0.0222388563182584, -0.020322184923873, 0.017783917715061, -0.64

# 3.  Prediction for regression

In [35]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress',               -- Model table
         'lin_housing',               -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT *, ABS(y-estimated_y) as abs_diff FROM lin_housing JOIN mlp_regress_prediction USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,x,zipcode,y,estimated_y,abs_diff
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0,24.0157610175,0.0157610174549
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6,21.6066799522,0.00667995224621
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7,34.7049026219,0.00490262186459
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4,33.4594187804,0.0594187804031
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2,36.1274360376,0.0725639623819
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7,28.7107840628,0.0107840627703
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9,22.8131646352,0.0868353648218
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1,27.0359341539,0.0640658460926
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5,16.5433763557,0.0433763557018
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9,18.300082072,0.599917928016


RMS error:

In [37]:
%%sql
SELECT SQRT(AVG((y-estimated_y)*(y-estimated_y))) as rms_error FROM lin_housing 
JOIN mlp_regress_prediction USING (id);

1 rows affected.


rms_error
0.227864472932


# Regression with Grouping and Mini-Batching

# 1. Call mini-batch preprocessor
To use grouping and mini-batching, we must first re-run the preprocessor and specify grouping by zipcode (i.e., grouping must exactly match between mini-batch preprocessor and training function).

In [38]:
%%sql
DROP TABLE IF EXISTS lin_housing_packed, lin_housing_packed_summary, lin_housing_packed_standardization;

SELECT madlib.minibatch_preprocessor('lin_housing',         -- Source table
                                     'lin_housing_packed',  -- Output table
                                     'y',                   -- Dependent variable
                                     'x',                   -- Independent variables
                                     'zipcode'              -- Group by zipcode
                                     );

Done.
1 rows affected.


minibatch_preprocessor
""


# 2. Train regression model with grouping and mini-batching

In [39]:
%%sql
DROP TABLE IF EXISTS mlp_regress_group, mlp_regress_group_summary, mlp_regress_group_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_regression(
    'lin_housing_packed',    -- Source table
    'mlp_regress_group',    -- Desination table
    'independent_varname',              -- Input features
    'dependent_varname',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE,            -- Not verbose
    'zipcode'         -- Grouping column
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


View regression model with grouping:

In [40]:
%%sql
SELECT * FROM mlp_regress_group;

2 rows affected.


[(20001, [-0.0541461766085941, -0.0963807051813863, -0.0848363366668209, -0.0319143812339262, -0.020761661615561, -0.0605320681822323, -0.0162076728281467, 0.0227860438636339, -0.0916700250380406, -0.111362351574317, 0.0377391298665158, 0.0127007527062157, -0.0952581387665948, -0.214512528119903, 0.0718202819562677, 0.0645897465081911, -0.0281239845111408, 0.0613106606121158, -0.110773685039869, 0.0493427184575546, 0.0554531204735584, 0.0549208883866061, 0.0911719681403288, 0.0845051558752586, 0.041591936434053, -0.0213147530256813, -0.119120080131304, 0.218267102509466, 0.0925560376819976, -0.041619620839362, -0.10417267057653, -0.0434034162785702, 0.117308065670177, 0.0391002196276418, -0.0834852818107183, -0.0610226764071191, -0.0381868116077067, 0.118712393283794, -0.00650355905397435, -0.0532426608718398, -0.0286900733831179, -0.015797135368718, -0.100222330376634, 0.117574039614028, -0.184418480416043, -0.0798476306725336, -0.138747104681873, -0.0264182483960586, 0.000190512539336849, -0.0202787871381347, -0.0395923923412633, -0.0221753750874551, 0.0135959002066061, -0.000869079545667455, -0.0848416946473226, 0.0660949306612554, -0.134177782967401, -0.160765841890361, -0.028550055944261, -0.111437498802393, 0.217703057316866, 0.0938248088614894, -0.109631302168519, 0.1080756672649, -0.15447783339759, 0.10469866154726, -0.118765499240873, -0.156385210082563, -0.148746460790189, -0.0647877262242873, 0.14915924905368, 0.12438692634196, -0.128796474371223, -0.0130461106093999, 0.0689913582977552, 0.011534075692911, -0.0748234571350883, -0.0243578971786008, 0.0276566429968495, -0.105599088159167, 0.0285231010084415, -0.0832117551394463, 0.116172056727386, -0.0116878240983329, -0.0447057668126281, 0.0710695299380156, -0.103253371000572, -0.0333979729097441, 0.115154756283138, 0.113997360277011, 0.146786065166738, 0.000583310671245868, 0.217462907461374, -0.017070768918925, 0.0994259647109968, -0.0739620858824955, 0.0170735392737714, -0.0719017083084897, -0.342465735901241, 0.0276196379610093, -0.0946000247247486, -0.11739418723803, 0.0826197524513089, -0.0314249168487506, -0.381209992338566, 0.355031351822605, 0.13822349231823, 0.041976043385705, -0.261552754045386, 0.341363462992429, -0.0397718893622874, 0.189213008213591, 0.156481063738914, -0.0469719169427159, -0.00169602913702448, -0.37804392652785, -0.312270012864678, 0.309163126859515, 0.19213763935615, -0.254288640659383, 0.107521714304294, 0.0825580238753104, 0.0962195479152872, -0.134840008084838, 0.0119453067009529, 0.0348756856377711, -0.0274248201008274, -0.152275074753546, -0.029942720340659, -0.0566871749116289, -0.225882797939311, -0.261168719211093, -0.039471303836738, 0.0782214266340984, -0.128485340758724, 0.0819043493100628, 0.0313323688844088, -0.189551179530182, 0.118913910106624, -0.132971056366972, -0.164941615044348, -0.105020619432348, 0.14338000423215, -0.00292783358800445, -0.125725718185904, 0.233530408774304, 0.0390016244963848, -0.13573803491991, 0.04307399009858, 0.159970184826762, -0.0952204271329933, -0.0419644872907085, -0.00018261572632825, 0.028944988498512, -0.119258285289917, -0.039622114509471, -0.00246417966040517, 0.0143467118676396, -0.11769433227617, 0.0752643577550166, -0.00693603462153328, -0.0591130547012542, 0.0370456966008088, 0.018813502166078, 0.035694802650666, -0.152253920594493, 0.022521127007706, -0.0483011379432575, -0.0336579828615425, 0.0565386052073328, 0.120587417509158, -0.0099043679042341, 0.0409871497931495, 0.0580622102996774, -0.0834756761716336, 0.0976168374845424, -0.00406257613923814, 0.0399634958708954, -0.125019131778949, -0.00780777524156062, 0.081136124673015, 0.0478216197444689, -0.0889362434015769, 0.147811557690072, -0.0539336794034292, -0.0815832840814789, 0.000236448453410446, 0.0352969905854623, -0.232796504856402, -0.134289827733899, 0.00940088353316586, 0.120317706562595, -0.081766376273438, -0.186556798386095, 0.237040316580706, -0.0822971428706008, -0.119699789700703, -0.0931646851382498, 0.117569

# 3. Prediction for regression with grouping and mini-batching

In [42]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress_group',         -- Model table
         'lin_housing',             -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT * FROM lin_housing JOIN mlp_regress_prediction USING (zipcode, id) ORDER BY zipcode, id;

Done.
1 rows affected.
20 rows affected.


zipcode,id,x,y,estimated_y
20001,12,"[Decimal('1'), Decimal('0.11747'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0090'), Decimal('82.90'), Decimal('6.2267'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('13.27')]",18.9,18.8921920465
20001,13,"[Decimal('1'), Decimal('0.09378'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.8890'), Decimal('39.00'), Decimal('5.4509'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('390.50'), Decimal('15.71')]",21.7,21.7015748487
20001,14,"[Decimal('1'), Decimal('0.62976'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9490'), Decimal('61.80'), Decimal('4.7075'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('396.90'), Decimal('8.26')]",20.4,20.2487092793
20001,15,"[Decimal('1'), Decimal('0.63796'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('6.0960'), Decimal('84.50'), Decimal('4.4619'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('380.02'), Decimal('10.26')]",18.2,18.2797485487
20001,16,"[Decimal('1'), Decimal('0.62739'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.8340'), Decimal('56.50'), Decimal('4.4986'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('395.62'), Decimal('8.47')]",19.9,20.0149898814
20001,17,"[Decimal('1'), Decimal('1.05393'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9350'), Decimal('29.30'), Decimal('4.4986'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('386.85'), Decimal('6.58')]",23.1,23.1046177051
20001,18,"[Decimal('1'), Decimal('0.78420'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9900'), Decimal('81.70'), Decimal('4.2579'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('386.75'), Decimal('14.67')]",17.5,17.493668219
20001,19,"[Decimal('1'), Decimal('0.80271'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.4560'), Decimal('36.60'), Decimal('3.7965'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('288.99'), Decimal('11.69')]",20.2,20.1905773915
20001,20,"[Decimal('1'), Decimal('0.72580'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.7270'), Decimal('69.50'), Decimal('3.7965'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('390.95'), Decimal('11.28')]",18.2,18.1658904889
94016,1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",24.0,23.9996342128
